<a href="https://colab.research.google.com/github/B0BWAX/AMEX-DEFAULT-PREDICTION/blob/main/AMEX_DT_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# using kaggle API
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/API-KEYS/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
# downloading modified dataset from last notebook
!kaggle datasets download -d bobwax/amex-agg-dataset
!unzip amex-agg-dataset.zip

 98% 889M/905M [00:24<00:00, 62.2MB/s]
100% 905M/905M [00:24<00:00, 38.4MB/s]
Archive:  amex-agg-dataset.zip
  inflating: amex_agg_data.csv       


In [4]:
# downloading GPU library
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 476, done.
remote: Counting objects: 100% (207/207), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 476 (delta 141), reused 124 (delta 91), pack-reused 269
Receiving objects: 100% (476/476), 131.59 KiB | 7.31 MiB/s, done.
Resolving deltas: 100% (243/243), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.5 MB/s eta 0:00:00
***********************************************************************
Woo! Your instance has a Tesla T4 GPU!
We will install the latest stable RAPIDS via pip 24.4.*!  Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.3/473.3 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 861.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 GB 1.

In [5]:
import cuml, cudf # GPU libiraries
import pandas as pd, numpy as np # CPU libiraries
import pickle
import gc

In [6]:
from cuml.model_selection import train_test_split

In [7]:
# importing data
data = cudf.read_csv('/content/amex_agg_data.csv')

In [8]:
data.head()

,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,D_39_last,...,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique,target
0,0.933824,0.024194,0.868580,0.960384,0.934745,0.230769,0.832050,0,3,0,...,13,0,1,13,-1,1,13,6,1,0
1,0.899820,0.022119,0.861109,0.929122,0.880519,7.153846,6.743468,0,19,6,...,13,0,1,13,-1,1,13,6,1,0
2,0.878454,0.028911,0.797670,0.904482,0.880875,0.000000,0.000000,0,0,0,...,13,2,1,13,-1,1,13,6,1,0
3,0.598969,0.020107,0.567442,0.623392,0.621776,1.538462,3.017046,0,9,0,...,13,0,1,13,-1,1,13,3,3,0
4,0.891679,0.042325,0.805045,0.940382,0.871900,0.000000,0.000000,0,0,0,...,13,0,1,13,1,1,13,6,1,0


In [9]:
# splitting data
X = data.drop(columns=['target'])
y = data['target']
del data # no longer needed
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
del X, y # no longer needed

In [10]:
gc.collect() # free up GPU RAM

7392

### Metric
The metric used to compare between models will be the competition metric which is:

>The evaluation metric, M, for this competition is the mean of two measures of rank ordering: Normalized Gini Coefficient, $G$, and default rate captured at 4%, $D$.
>$$M=0.5⋅(G+D)$$
>The default rate captured at 4% is the percentage of the positive labels (defaults) captured within the highest-ranked 4% of the predictions, and represents a Sensitivity/Recall statistic.
>For both of the sub-metrics $G$ and $D$ , the negative labels are given a weight of 20 to adjust for downsampling.
>This metric has a maximum value of 1.0.

In [11]:
def amex_metric_mod(y_true, y_pred):

    labels     = np.transpose(np.array([y_true, y_pred]))
    labels     = labels[labels[:, 1].argsort()[::-1]]
    weights    = np.where(labels[:,0]==0, 20, 1)
    cut_vals   = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four   = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])

    gini = [0,0]
    for i in [1,0]:
        labels         = np.transpose(np.array([y_true, y_pred]))
        labels         = labels[labels[:, i].argsort()[::-1]]
        weight         = np.where(labels[:,0]==0, 20, 1)
        weight_random  = np.cumsum(weight / np.sum(weight))
        total_pos      = np.sum(labels[:, 0] *  weight)
        cum_pos_found  = np.cumsum(labels[:, 0] * weight)
        lorentz        = cum_pos_found / total_pos
        gini[i]        = np.sum((lorentz - weight_random) * weight)

    return 0.5 * (gini[1]/gini[0] + top_four)

*This metric is very similar to ROC AUC score

*Accuracy score also computed but it's important to note that accuracy alone is not a good evaluation metric in this case.

## Model

In [12]:
from cuml import RandomForestClassifier

In [13]:
rf_model = RandomForestClassifier( max_depth=10,
              n_estimators=100,
              random_state=0)

/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)


In [14]:
rf_model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/cupy/cuda/compiler.py:233: PerformanceWarning: Jitify is performing a one-time only warm-up to populate the persistent cache, this may take a few seconds and will be improved in a future release...
  jitify._init_module()
/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


In [18]:
with open('AMEX_rf_model.pkl', 'wb') as f: # save model
    pickle.dump(rf_model, f)

In [15]:
predictions = rf_model.predict(X_test)

In [16]:
rf_comp = amex_metric_mod( y_test.to_numpy(), predictions.to_numpy())
rf_accuracy = cuml.metrics.accuracy_score(y_test, predictions)
print(f" Random Forest Scores \n ------------------------------------- \n Accuracy: {rf_accuracy} \n Competition Metric: {rf_comp}")

 Random Forest Scores 
 ------------------------------------- 
 Accuracy: 0.8906394243240356 
 Competition Metric: 0.541701129757983
